In [ ]:
from time import time

from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.python.keras.layers import Input, LSTM, Dense, Activation, Dropout
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.optimizers import Nadam
from tensorflow.python.keras.utils import to_categorical
import numpy as np


timestamp = int(time())

## Dados

Carregamos todo o texto (porque dá) e passamos para minúsculas

In [ ]:
text = open('data/casmurro.txt', encoding='utf-8').read()
text = text.lower()

Detectamos os caracteres e mapeamos os índices

In [ ]:
chars = sorted(set(text))
char_to_int = {c: i for i, c in enumerate(chars)}

Geramos os exemplos "sequencia -> proxima letra"

In [ ]:
length = 32
x, y = [], []
word = text[:length]
for letter in text[length:]:
    x.append([char_to_int[w] for w in word])
    y.append(char_to_int[letter])
    word = word[1:] + letter
x = np.reshape(x, (-1, length, 1)) / len(chars)
y = to_categorical(y, len(chars))

Embaralhando o dataset

In [ ]:
i = np.random.permutation(x.shape[0])
x, y = x[i], y[i]

Separando treino e teste

In [ ]:
i = int(.9 * len(x))
x_train, x_test = x[:i], x[i:]
y_train, y_test = y[:i], y[i:]

## Modelo

Camada de entrada (compatível com a forma de x)

In [ ]:
out = entry = Input(shape=x_train.shape[1:])

Camada de memória

In [ ]:
out = LSTM(32)(out)

Camada de saída com um neurônio para cada caractere e aplicação do softmax para obtermos uma distribuição de probabilidade

In [ ]:
out = Dropout(0.5)(out)
out = Dense(y_train.shape[1])(out)
out = Activation('softmax')(out)

Definição do modelo em si

In [ ]:
net = Model(entry, out)

Imprimimos a descrição do modelo

In [ ]:
net.summary()

## Treinamento

Definição do custo e da otimização

In [ ]:
net.compile(
    loss='categorical_crossentropy',
    optimizer=Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004),
    metrics=['accuracy'])

Treinamento em si

In [ ]:
net.fit(
    x_train, y_train,
    batch_size=512,
    initial_epoch=10, epochs=20,
    validation_data=(x_test, y_test),
    callbacks=[
        ModelCheckpoint('save/text.{epoch:02d}.h5'),
        TensorBoard(log_dir='logs/text_{}'.format(timestamp), histogram_freq=1)])

## Geração de texto

Ponto de partida

In [ ]:
net = load_model('save/text.01.h5')

In [ ]:
text = '''Era uma vez'''.lower()
text = '''( function( global, factory ) { '''.lower()

Geração em si

In [ ]:
for _ in range(1000):
    word = text[-length:]

    x = [char_to_int[w] for w in word]
    x = np.reshape(x, (-1, length, 1)) / len(chars)
    y = net.predict(x, verbose=0)[0]

    char = np.random.choice(chars, p=y)
    # char = chars[np.argmax(y)]

    text += char

print(text)